Importing primary dependencies for data fetch and cleanup

In [77]:
import pandas as pd
import numpy as np
import re

from pandas import read_csv

Various constants and data fetch

In [107]:
max_features=7200
max_len=1024
test_prop=0.3
epoch_count=100
batch_size=512

In [79]:
fake=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/fake.csv')
real=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/true.csv')
data2=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/cleaned/ds2.csv')

In [80]:
fake.sample(4)

,title,text,subject,date
18939,BREAKING: ELECTRONICS BANNED on Some U.S. Flig...,COULD IT BE THAT EXPERT BOMB MAKER Ibrahim al-...,left-news,"Mar 21, 2017"
2883,GOP Lawmaker Deactivates Twitter Account Afte...,A Nebraska State Senator was busted previously...,News,"January 23, 2017"
7633,West Virginia Republicans Declare War On Poli...,In a move that shows just how little Republica...,News,"March 7, 2016"
10247,CNN’S JIM ACOSTA Schooled on the Meaning of th...,We wish President Trump could clone SENIOR ADV...,politics,"Aug 2, 2017"


In [81]:
real.sample(4)

,title,text,subject,date
15152,"We'll buy arms from Russia, Philippines' Duter...","DANANG, Vietnam (Reuters) - Philippine Preside...",worldnews,"November 10, 2017"
9070,"Biden to rebuke Trump foreign policy ideas, de...",WASHINGTON (Reuters) - Vice President Joe Bide...,politicsNews,"June 20, 2016"
15884,Mexico women protest violence in skeletal Day ...,MEXICO CITY (Reuters) - Wearing black clothing...,worldnews,"November 2, 2017"
12418,Russia will keep bases in Syria to strike at i...,MOSCOW (Reuters) - Russia will keep a naval an...,worldnews,"December 12, 2017"


Compressed every cleanup job into the *clean_csv* function

In [82]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string
def clean_csv(df,identity): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df['identity']=identity #add the identity (T/F) or (0/1) in a new attribute to the data
    df.reset_index() #reset all indexes of the dataset
    print('Cleaning completed.')
    return df

In [83]:
fake1=fake.iloc[:int(len(fake)/2),:]
fake2=fake.iloc[int(len(fake)/2):,:]

In [84]:
fake1=clean_csv(fake1,0)

Cleaning CSV.
Cleaning completed.


In [85]:
fake2=clean_csv(fake2,0)

Cleaning CSV.
Cleaning completed.


In [86]:
fake=pd.concat([fake1,fake2])

In [87]:
fake.sample(4)

,title,text,subject,date,length,identity
21413,WOW! WORLD’S TOP PHYSICIST AND DEMOCRAT: Obama...,wow worlds top physicist and democrat obama ba...,left-news,"Oct 15, 2015",6486,0
14732,SWISS ARMY CHIEF WARNS CITIZENS About Explosiv...,swiss army chief warns citizens about explosiv...,politics,"Dec 31, 2015",1894,0
23045,Trump Swings Back at Author of Fake Dossier: ‘...,trump swings back at author of fake dossier fa...,Middle-east,"January 19, 2017",1870,0
11086,BREAKING NEWS: SECOND MUSLIM Doctor And Wife A...,breaking news second muslim doctor and wife ar...,politics,"Apr 21, 2017",1755,0


In [88]:
real1=real.iloc[:int(len(real)/2),:]
real2=real.iloc[int(len(real)/2):,:]

In [89]:
real1=clean_csv(real1,1)

Cleaning CSV.
Cleaning completed.


In [90]:
real2=clean_csv(real2,1)

Cleaning CSV.
Cleaning completed.


In [91]:
real=pd.concat([real1,real2])

In [92]:
real.sample(4)

,title,text,subject,date,length,identity
16648,Six crew from German ship kidnapped in Nigeria...,six crew from german ship kidnapped in nigeria...,worldnews,"October 24, 2017",1936,1
398,House intel panel advances NSA spying bill des...,house intel panel advances nsa spying bill des...,politicsNews,"December 1, 2017",2730,1
17674,Migrant deaths in the Sahara likely twice Medi...,migrant deaths in the sahara likely twice medi...,worldnews,"October 12, 2017",2566,1
5960,Republican senator wants Russia sanctions put ...,republican senator wants russia sanctions put ...,politicsNews,"January 27, 2017",530,1


Dropping all unnecessary columns

In [93]:
fake=fake[['text','identity']]
real=real[['text','identity']]

In [94]:
fake.sample(2)

,text,identity
14373,invasion of america possibility of a trump wal...,0
15564,obamas dream is americas nightmare 121 illegal...,0


In [95]:
data1=pd.concat([real,fake],ignore_index=True)

In [96]:
data1.tail(4)

,text,identity
43748,justice yahoo settles email privacy classactio...,0
43749,sunnistan us and allied safe zone plan to take...,0
43750,how to blow 700 million al jazeera america fin...,0
43751,10 us navy sailors held by iranian military s...,0


In [97]:
data=pd.concat([data1,data2],ignore_index=True)

In [98]:
data.tail(4)

,text,identity,Unnamed: 0
61810,nfl playoffs schedule matchups and odds the n...,0,18058.0
61811,macys is said to receive takeover approach by ...,0,18059.0
61812,nato russia to hold parallel exercises in balk...,1,18060.0
61813,what keeps the f35 alive david swanson is an...,1,18061.0


In [99]:
data=data[['text','identity']]
data.reset_index()
data.tail()

,text,identity
61809,rapper ti trump a poster child for white supre...,0
61810,nfl playoffs schedule matchups and odds the n...,0
61811,macys is said to receive takeover approach by ...,0
61812,nato russia to hold parallel exercises in balk...,1
61813,what keeps the f35 alive david swanson is an...,1


Importing dependencies for ANN and Representation

In [100]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,Dropout,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

Tokenizing the dataset for use in the ANN

In [101]:
def tokenize_df(df,max_features,max_len): #tokenize the df using keras tokenizer
        print('Tokenizing DF.')
        tokenizer=Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split =' ') #filters is redundant, but still keeping if I missed any symbols previously
        tokenizer.fit_on_texts(texts=df['text']) #fit
        X=tokenizer.texts_to_sequences(texts=df['text'])
        X=pad_sequences(sequences=X,maxlen=max_len,padding='pre') #padding the set of words
        Y=df['identity'].values #identities (0/1)
        V=tokenizer.word_index #vocabulary
        print('Tokenized DF.')
        return X,Y,V

In [102]:
X,Y,V=tokenize_df(data,max_features,max_len)

Tokenizing DF.
Tokenized DF.


Creating the model and compiling it to be used.

In [103]:
def create_lstm_model(max_features): #creating the internal structure of the simple LSTM model
        print('Creating LSTM model.')
        lstm_model=Sequential(name='Sierra')
        lstm_model.add(layer=Embedding(input_dim=max_features,output_dim=128,name='Echo'))
        lstm_model.add(layer=LSTM(units=128,name='Lima'))
        lstm_model.add(layer=Dense(1,activation='sigmoid',name ='Omega'))
        print('Model creation completed.')
        lstm_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        print('Model compiled.')
        return lstm_model

In [104]:
lstm_model=create_lstm_model(max_features)

Creating LSTM model.
Model creation completed.
Model compiled.


In [105]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

In [108]:
lstm_model.fit(X_train,Y_train,batch_size=batch_size,epochs=epoch_count,verbose=1,validation_split=test_prop,shuffle=1,use_multiprocessing=True)

Epoch 1/100
64/64 [==============================] - 13s 201ms/step - loss: 0.3189 - accuracy: 0.8455 - val_loss: 0.4127 - val_accuracy: 0.7918
Epoch 2/100
64/64 [==============================] - 14s 215ms/step - loss: 0.2990 - accuracy: 0.8633 - val_loss: 0.3762 - val_accuracy: 0.8196
Epoch 3/100
64/64 [==============================] - 14s 218ms/step - loss: 0.2286 - accuracy: 0.9029 - val_loss: 0.2868 - val_accuracy: 0.8942
Epoch 4/100
64/64 [==============================] - 14s 220ms/step - loss: 0.1188 - accuracy: 0.9563 - val_loss: 0.2275 - val_accuracy: 0.9168
Epoch 5/100
64/64 [==============================] - 13s 207ms/step - loss: 0.0893 - accuracy: 0.9699 - val_loss: 0.1810 - val_accuracy: 0.9454
Epoch 6/100
64/64 [==============================] - 13s 210ms/step - loss: 0.0669 - accuracy: 0.9790 - val_loss: 0.1835 - val_accuracy: 0.9400
Epoch 7/100
64/64 [==============================] - 13s 210ms/step - loss: 0.0805 - accuracy: 0.9737 - val_loss: 0.2235 - val_accuracy:

In [109]:
Y_pred=(lstm_model.predict(X_test)>=0.5).astype(int)

483/483 [==============================] - 7s 13ms/step


In [110]:
accuracy_score(Y_test,Y_pred)

0.9436391872654329

In [112]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      8209
           1       0.95      0.93      0.94      7245

    accuracy                           0.94     15454
   macro avg       0.94      0.94      0.94     15454
weighted avg       0.94      0.94      0.94     15454



In [113]:
lstm_model.save('models/ds1-2_combined_epoch-100')

In [115]:
nmod=tf.keras.saving.load_model('models/ds1-2_combined_epoch-100')

In [116]:
y_pred = (nmod.predict(X_test) >=0.5).astype(int)

483/483 [==============================] - 7s 13ms/step


In [117]:
accuracy_score(Y_test,y_pred)

0.9436391872654329